# 古诗词对子生成

## Step1 训练数据查看

In [1]:
# 使用古诗来作为例子，读取数据，看看长什么样子
with open('./char_dataset/poetry.txt', 'r') as f:
    poetry_corpus = f.read()

In [2]:
print poetry_corpus[:33]

寒随穷律变，春逐鸟声开


In [3]:
# 为了可视化比较方便，我们将一些其他字符替换成空格
poetry_corpus = poetry_corpus.replace('\n', '').replace('\r', '').replace('，', ' ').replace('。', ' ')
print poetry_corpus.decode('utf-8')[:12]

寒随穷律变 春逐鸟声开 


In [4]:
poetry_corpus = poetry_corpus.decode('utf-8')

In [5]:
# 看看字符数
len(poetry_corpus)

72000

## Step2 训练数据处理

In [6]:
# 文本数值表示, 将文字转换成数字，对所有非重复的字符，可以从 0 开始建立索引
# 同时为了节省内存的开销，可以词频比较低的字去掉
import numpy as np

class TextConverter(object):
    def __init__(self, text_path, max_vocab=5000):
        """
        建立一个字符索引转换器
        Args:
            text_path: 文本位置
            max_vocab: 最大的单词数量
        """
        
        with open(text_path, 'r') as f:
            tt = f.read()
        tt = tt.replace('\n', '').replace('\r', '').replace('，', ' ').replace('。', ' ')
        text = tt.decode('utf-8')
        
        # 去掉重复的字符
        vocab = set(text)

        # 如果单词总数超过最大数值，去掉频率最低的
        vocab_count = {}
        
        # 计算单词出现频率并排序
        for word in vocab:
            vocab_count[word] = 0
        for word in text:
            vocab_count[word] += 1
        vocab_count_list = []
        for word in vocab_count:
            vocab_count_list.append((word, vocab_count[word]))
        vocab_count_list.sort(key=lambda x: x[1], reverse=True)
        
        # 如果超过最大值，截取频率最低的字符
        if len(vocab_count_list) > max_vocab:
            vocab_count_list = vocab_count_list[:max_vocab]
        vocab = [x[0] for x in vocab_count_list]
        self.vocab = vocab

        self.word_to_int_table = {c: i for i, c in enumerate(self.vocab)}
        self.int_to_word_table = dict(enumerate(self.vocab))

    @property
    def vocab_size(self):
        return len(self.vocab) + 1

    def word_to_int(self, word):
        if word in self.word_to_int_table:
            return self.word_to_int_table[word]
        else:
            return len(self.vocab)

    def int_to_word(self, index):
        if index == len(self.vocab):
            return '<unk>'
        elif index < len(self.vocab):
            return self.int_to_word_table[index]
        else:
            raise Exception('Unknown index!')

    def text_to_arr(self, text):
        arr = []
        for word in text:
            arr.append(self.word_to_int(word))
        return np.array(arr)

    def arr_to_text(self, arr):
        words = []
        for index in arr:
            words.append(self.int_to_word(index))
        return "".join(words)

In [7]:
convert = TextConverter('./char_dataset/poetry.txt', max_vocab=10000)

In [8]:
# 可以可视化一下数字表示的字符
# 原始文本字符
txt_char = poetry_corpus[:12]
print(txt_char)

# 转化成数字
print(convert.text_to_arr(txt_char))

寒随穷律变 春逐鸟声开 
[ 74 155 415 739 380   0   6 189 100  79  31   0]


## Step3 构造训练样本

In [9]:
# 构造时序样本数据
n_step = 60

# 总的序列个数
num_seq = int(len(poetry_corpus) / n_step)

# 去掉最后不足一个序列长度的部分
text = poetry_corpus[:num_seq*n_step]

print(num_seq)

1200


In [10]:
import torch

# 将序列中所有的文字转换成数字表示，重新排列成 (num_seq x n_step) 的矩阵
arr = convert.text_to_arr(text)
print(arr[15:30])

arr = arr.reshape((num_seq, -1))
arr = torch.from_numpy(arr)

print(arr.shape)
print(arr[1, :])

[235 108   0 104 120 263   4 299   0 248  68  90 208 292   0]
torch.Size([1200, 60])

    3
 2598
  206
    4
   58
    0
    1
  145
   37
   68
  172
    0
  149
  518
 1194
    2
  119
    0
  251
  100
   79
   39
  670
    0
  394
  156
 2981
  300
  784
    0
   47
 1444
  275
  177
 1020
    0
   30
   34
    6
  541
  792
    0
  464
   42
  100
    4
 1373
    0
  247
   97
  127
  757
   35
    0
  217
  153
   67
  171
   71
    0
[torch.LongTensor of size 60]



In [11]:
# 据此，我们可以构建 PyTorch 中的数据读取来训练网络，
# 这里我们将最后一个字符的输出 label 定为输入的第一个字符，也就是"床前明月光"的输出是"前明月光床"
class TextDataset(object):
    def __init__(self, arr):
        self.arr = arr
        
    def __getitem__(self, item):
        x = self.arr[item, :]
        
        # 构造 label
        y = torch.zeros(x.shape)
        # 将输入的第一个字符作为最后一个输入的 label
        y[:-1], y[-1] = x[1:], x[0]
        #y[:-3],y[-3:] = x[3:],x[:3]
        
        return x, y
    
    def __len__(self):
        return self.arr.shape[0]

In [12]:
train_set = TextDataset(arr)

我们可以取出其中一个数据集参看一下是否是我们描述的这样

In [13]:
# 可以取出其中一个数据,看一下是否是描述的这样
x, y = train_set[1]
print(convert.arr_to_text(x.numpy()))
print(convert.arr_to_text(y.numpy()))

日晃百花色 风动千林翠 池鱼跃不同 园鸟声还异 寄言博通者 知予物外志 一朝春夏改 隔夜鸟花迁 阴阳深浅叶 晓夕重轻烟 
晃百花色 风动千林翠 池鱼跃不同 园鸟声还异 寄言博通者 知予物外志 一朝春夏改 隔夜鸟花迁 阴阳深浅叶 晓夕重轻烟 日


## Step4 构造模型

In [14]:
from torch import nn
from torch.autograd import Variable

use_gpu = False

# 模型可以定义成非常简单的三层，第一层是词嵌入，第二层是 RNN 层，
# 因为最后是一个分类问题，所以第三层是线性层，最后输出预测的字符
class CharRNN(nn.Module):
    def __init__(self, num_classes, embed_dim, hidden_size, 
                 num_layers, dropout):
        super(CharRNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.word_to_vec = nn.Embedding(num_classes, embed_dim)
        self.rnn = nn.GRU(embed_dim, hidden_size, num_layers, dropout)
        #self.rnn = nn.LSTM(embed_dim, hidden_size, num_layers, dropout)
        self.project = nn.Linear(hidden_size, num_classes)

    def forward(self, x, hs=None):
        batch = x.shape[0]
        if hs is None:
            hs = Variable(
                torch.zeros(self.num_layers, batch, self.hidden_size))
            if use_gpu:
                hs = hs.cuda()
        word_embed = self.word_to_vec(x)  # (batch, len, embed)
        word_embed = word_embed.permute(1, 0, 2)  # (len, batch, embed)
        out, h0 = self.rnn(word_embed, hs)  # (len, batch, hidden)
        le, mb, hd = out.shape
        out = out.view(le * mb, hd)
        out = self.project(out)
        out = out.view(le, mb, -1)
        out = out.permute(1, 0, 2).contiguous()  # (batch, len, hidden)
        return out.view(-1, out.shape[2]), h0

## Step5 训练模型

在训练模型的时候，我们知道这是一个分类问题，所以可以使用交叉熵作为 loss 函数，在语言模型中，我们通常使用一个新的指标来评估结果，这个指标叫做困惑度(perplexity)，可以简单地看作对交叉熵取指数，这样其范围就是 $[1, +\infty]$，也是越小越好。

另外，我们前面讲过 RNN 存在着梯度爆炸的问题，所以我们需要进行梯度裁剪，在 pytorch 中使用 `torch.nn.utils.clip_grad_norm` 就能够轻松实现

In [15]:
from torch.utils.data import DataLoader

batch_size = 128
train_data = DataLoader(train_set, batch_size, True, num_workers=4)

In [16]:
from util.trainer import ScheduledOptim

model = CharRNN(convert.vocab_size, 512, 512, 2, 0.5)
if use_gpu:
    model = model.cuda()
criterion = nn.CrossEntropyLoss()

basic_optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
optimizer = ScheduledOptim(basic_optimizer)

In [17]:
epochs = 30
for e in range(epochs):
    step_nu = 0
    train_loss = 0
    for data in train_data:
        step_nu += 1
        if (step_nu%5 == 0):
            print step_nu
        x, y = data
        x = x.long()
        y = y.long()
        if use_gpu:
            x = x.cuda()
            y = y.cuda()
        x, y = Variable(x), Variable(y)

        # Forward.
        score, _ = model(x)
        loss = criterion(score, y.view(-1))

        # Backward.
        optimizer.zero_grad()
        loss.backward()
        # Clip gradient.
        nn.utils.clip_grad_norm(model.parameters(), 5)
        optimizer.step()

        train_loss += loss.data[0]
    print('epoch: {}, perplexity is: {:.3f}, lr:{:.1e}'.format(e+1, np.exp(train_loss / len(train_data)), optimizer.lr))

5
10
epoch: 1, perplexity is: 1208.366, lr:1.0e-03
5
10
epoch: 2, perplexity is: 587.341, lr:1.0e-03
5
10
epoch: 3, perplexity is: 475.234, lr:1.0e-03
5
10
epoch: 4, perplexity is: 406.376, lr:1.0e-03
5
10
epoch: 5, perplexity is: 372.025, lr:1.0e-03
5
10
epoch: 6, perplexity is: 348.377, lr:1.0e-03
5
10
epoch: 7, perplexity is: 326.711, lr:1.0e-03
5
10
epoch: 8, perplexity is: 307.925, lr:1.0e-03
5
10
epoch: 9, perplexity is: 288.974, lr:1.0e-03
5
10
epoch: 10, perplexity is: 264.509, lr:1.0e-03
5
10
epoch: 11, perplexity is: 243.038, lr:1.0e-03
5
10
epoch: 12, perplexity is: 218.856, lr:1.0e-03
5
10
epoch: 13, perplexity is: 195.536, lr:1.0e-03
5
10
epoch: 14, perplexity is: 173.147, lr:1.0e-03
5
10
epoch: 15, perplexity is: 150.774, lr:1.0e-03
5
10
epoch: 16, perplexity is: 130.804, lr:1.0e-03
5
10
epoch: 17, perplexity is: 112.872, lr:1.0e-03
5
10
epoch: 18, perplexity is: 96.170, lr:1.0e-03
5
10
epoch: 19, perplexity is: 81.625, lr:1.0e-03
5
10
epoch: 20, perplexity is: 68.179, lr

## Step6 测试

In [18]:
# 生成文本的过程非常简单，前面已将讲过了，给定了开始的字符，然后不断向后生成字符，将生成的字符作为新的输入再传入网络。
# 这里需要注意的是，为了增加更多的随机性，我们会在预测的概率最高的前五个里面依据他们的概率来进行随机选择。

def pick_top_n(preds, top_n=5):
    top_pred_prob, top_pred_label = torch.topk(preds, top_n, 1)
    top_pred_prob /= torch.sum(top_pred_prob)
    top_pred_prob = top_pred_prob.squeeze(0).cpu().numpy()
    top_pred_label = top_pred_label.squeeze(0).cpu().numpy()
    c = np.random.choice(top_pred_label, size=1, p=top_pred_prob)
    return c

In [19]:
begin = '烟雨飘漫天'
#begin = '随风潜入夜'
begin = begin.decode('utf-8')
text_len = 36

model = model.eval()
samples = [convert.word_to_int(c) for c in begin]
input_txt = torch.LongTensor(samples)[None]
print input_txt


   71   144   341  3223    13
[torch.LongTensor of size 1x5]



In [20]:
if use_gpu:
    input_txt = input_txt.cuda()
input_txt = Variable(input_txt)
_, init_state = model(input_txt)
result = samples
model_input = input_txt[:, -1][:, None]
print model_input

Variable containing:
 13
[torch.LongTensor of size 1x1]



In [21]:
for i in range(text_len):
    out, init_state = model(model_input, init_state)
    pred = pick_top_n(out.data)
    model_input = Variable(torch.LongTensor(pred))[None]
    if use_gpu:
        model_input = model_input.cuda()
    result.append(pred[0])
    #result.append(out.data)

In [22]:
print result
text = convert.arr_to_text(result)
print text

[71, 144, 341, 3223, 13, 0, 9, 154, 275, 135, 20, 578, 0, 66, 149, 718, 33, 543, 0, 239, 446, 892, 75, 441, 1072, 0, 60, 155, 154, 134, 154, 1148, 161, 1148, 103, 103, 58, 145, 0, 1, 4]
烟雨飘漫天 山文物华空侧 芳池写清溪 岩峰势出轮质 影随文凤文短野短草草色动 风花
